In [1]:
!pip install transformers datasets scikit-learn --quiet

In [ ]:
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TextClassificationPipeline
from tqdm import tqdm
import numpy as np

df = pd.read_csv("/content/HateBRXplain.csv")
df = df.dropna(subset=["comment", "offensive_label"])  #Make sure there are no null values


In [ ]:
df = pd.read_csv("/content/HateBRXplain.csv")
print(df.columns)


Index(['id', 'comment', 'offensive_label', 'link_post',
       'rationales_annotator1', 'rationales_annotator2'],
      dtype='object')


In [ ]:
sensitive_terms = {
    "female_gender": ["mulher", "mulheres", "vagabunda", "quenga", "puta", "vadia", "piranha", "feminista", "bruxa", "ninfeta", "rapariga", "cachorra", "cachorrona", "quenga", "galinha", "cadela"],
    "male_gender": ["homem", "homens", "viado", "bicha", "maricas"],
    "race_ethnicity": ["preto", "negra", "negão", "branco", "racista", "macaco", "macaca", "chita",  "nordestino", "nordestinos", "judeu", "jornazista", "hipocristão", "macumbeira", "macumbeiro", "pastor"],
    "sexual_orientation": ["gay", "viado", "lésbica", "sapatão", "homossexual", "sapatona", "bicha", "veado"],
    "political_identity": ["comunista", "socialista", "esquerdista", "bolsonarista", "petista", "fascista", "petralha", "esquerda", "direita", "socialista", "bozo", "minions", "chauvinista", "petzadas", "ptzada", "pretalhas", "vermelhada"],
    "fatphobia": ["peppa", "pepa", "pepinha", "gorda", "gordo", "barriguda", "barrigudo", "porca", "porco", "roliça", "roliço", "vaca", "boi"],
    }


In [ ]:
def detect_sensitive(text):
    text = text.lower()
    for group, terms in sensitive_terms.items():
        if any(term in text for term in terms):
            return True
    return False

df["mentions_sensitive_group"] = df["comment"].apply(detect_sensitive)
print(f"Number of comments mentioning a sensitive group: {df['mentions_sensitive_group'].sum()}")



Number of comments mentioning a sensitive group: 1116


In [ ]:
device = 0 if torch.cuda.is_available() else -1

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)

pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=device)

# Obter probabilidades de classe 1 (discurso de ódio)
y_pred_probs = []
batch_size = 32

for i in tqdm(range(0, len(df), batch_size)):
    batch_texts = df["comment"].iloc[i:i+batch_size].tolist()
    outputs = pipeline(batch_texts)
    for out in outputs:
        y_pred_probs.append(out[1]["score"])  # score da classe 1

df["y_pred"] = y_pred_probs
y_true = df["offensive_label"].values
y_pred = df["y_pred"].values

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(

  5%|▍         | 10/219 [00:04<01:04,  3.25it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset

100%|██████████| 219/219 [01:12<00:00,  3.02it/s]


In [ ]:
def compute_subgroup_auc(y_true, y_pred, subgroup_mask):
    try:
        return roc_auc_score(y_true[subgroup_mask], y_pred[subgroup_mask])
    except:
        return np.nan

def compute_bpsn_auc(y_true, y_pred, subgroup_mask):
    # Background Positive (label==1, not subgroup) vs Subgroup Negative (label==0, subgroup)
    bpsn_mask = ((y_true == 1) & (~subgroup_mask)) | ((y_true == 0) & (subgroup_mask))
    return compute_subgroup_auc(y_true[bpsn_mask], y_pred[bpsn_mask], np.ones(bpsn_mask.sum(), dtype=bool))

def compute_bnsp_auc(y_true, y_pred, subgroup_mask):
    # Background Negative (label==0, not subgroup) vs Subgroup Positive (label==1, subgroup)
    bnsp_mask = ((y_true == 0) & (~subgroup_mask)) | ((y_true == 1) & (subgroup_mask))
    return compute_subgroup_auc(y_true[bnsp_mask], y_pred[bnsp_mask], np.ones(bnsp_mask.sum(), dtype=bool))

metrics = []

for group, terms in sensitive_terms.items():
    subgroup_mask = df["comment"].str.lower().apply(lambda x: any(term in x for term in terms))

    auc_subgroup = compute_subgroup_auc(y_true, y_pred, subgroup_mask)
    auc_bpsn = compute_bpsn_auc(y_true, y_pred, subgroup_mask)
    auc_bnsp = compute_bnsp_auc(y_true, y_pred, subgroup_mask)

    metrics.append({
        "group": group,
        "AUC_subgroup": auc_subgroup,
        "AUC_bpsn": auc_bpsn,
        "AUC_bnsp": auc_bnsp,
        "support": subgroup_mask.sum()
    })

df_metrics = pd.DataFrame(metrics)
print(df_metrics)

#save_results
df_metrics.to_csv("fairness_metrics.csv", index=False)
df[["comment", "offensive_label", "y_pred", "mentions_sensitive_group"]].to_csv("predictions_with_sensitive.csv", index=False)

                group  AUC_subgroup  AUC_bpsn  AUC_bnsp  support
0       female_gender      0.486362  0.525288  0.489945      420
1         male_gender      0.474937  0.573408  0.407647      127
2      race_ethnicity      0.686397  0.569432  0.634623      149
3  sexual_orientation      0.280000  0.285043  0.479256       15
4  political_identity      0.485154  0.481899  0.527652      444
5           fatphobia      0.499072  0.509839  0.504618       60
